# Efficiency - Add multiple points

| comparable algo list |
| - |
| pivot |
| delta |
| Monte Carlo |

In [1]:
import sys
sys.path.append("../..")

from sklearn import svm
import numpy as np
from copy import deepcopy

import src

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')

# size_choices = [10, 100, 1000, 10000]
size_choices = [10, 100, 200]
m_choices = [10, 15, 18, 20, 22, 25, 28, 30]
stone_m = 100

for size in size_choices:
    x_train, y_train, x_test, y_test, \
    columns_name = preprocess_data(
        't_train_' + str(size) + 'p.csv',
        't_test_' + str(size) + 'p.csv')

    # process data
    x_train_ = x_train[:-2]
    y_train_ = y_train[:-2]

    # for add
    add_points_x = x_train[-2:, :]
    add_points_y = y_train[-2:]

    x_test_ = x_test
    y_test_ = y_test

    union_d = generate_union_description(len(x_train), 10)
    union_d_ = deepcopy(union_d)
    for idx in [len(x_train), len(x_train) - 1]:
        for _ in union_d_:
            if idx in _:
                _.remove(idx)
                break

    # get a init ov
    init_ov = src.mc_owen(x_train_, y_train_, x_test_, y_test_, model, union_d_,
                               stone_m * len(y_train_), proc_num=2)
    save_npy('am_init_mc_plus_size' + str(size) + '.npy', init_ov)

    # set a stone
    mc_plus = src.mc_owen(x_train, y_train, x_test_, y_test_, model, union_d,
                               stone_m * len(y_train), proc_num=2)
    save_npy('am_mc_plus_size' + str(size) + '.npy', mc_plus)

    # mc -> 比较标准
    mc = src.mc_owen(x_train, y_train, x_test_, y_test_, model, union_d, stone_m * len(y_train))
    save_npy('am_mc'+ str(size) + 'm' + str(stone_m) + '.npy', mc)

    # half 需要提前算好的lov
    half_owen = src.PivotOwen(x_train_, y_train_, x_test_, y_test_, model,
                                  None, union_d_)
    lov = half_owen.prepare(m=len(y_train_)*stone_m)
    save_npy('am_half_lov_size' + str(size) + '.npy', lov)

    # # walk the m choices
    for m in m_choices:
        # delta
        delta_owen = src.DeltaOwen(x_train_, y_train_, x_test_, y_test_, model,
                                        init_ov, union_d_)

        m_delta_ov = delta_owen.add_single_point(add_points_x[0],
                                                 add_points_y[0],
                                                 4,
                                                 m=(len(y_train_)+1)*m)

        m_delta_ov = src.utils.eval_utility(x_train[:-1], y_train[-1], x_test_, y_test_, model)/ \
                     (np.sum(m_delta_ov)* m_delta_ov)
        
        union_d_delete_last = deepcopy(union_d)
        to_add = 0
        for i in len(union_d_delete_last): 
            if len(x_train_) in union_d_delete_last[i]:
                to_add = i
                union_d_delete_last[i].remove(len(x_train_))
                break  
        n_delta_owen = src.DeltaOwen(x_train[:-1], y_train[-1], x_test_, y_test_,
                                          model, m_delta_ov, union_d_delete_last)

        delta_ov = n_delta_owen.add_single_point(add_points_x[1],
                                                 add_points_y[1],
                                                 to_add,
                                                 m=(len(y_train_)+2)*m)
        save_npy('am_delta_size' + str(size) +'m' + str(m) + '.npy', delta_ov)

        # half
        half_owen = src.PivotOwen(x_train_, y_train_, x_test_, y_test_, model,
                                      None, union_d_)
        half_owen.lov = lov
        _ = half_owen.add_single_point(add_points_x[0], add_points_y[0], 4,
                                       (len(y_train_)+1)*m,
                                       {'flag_lov': True})
        half_owen.x_train = x_train[:-1,:]
        half_owen.y_train = y_train[:-1]
        half_owen.union_description = union_d_delete_last
        half_ov = half_owen.add_single_point(add_points_x[1], add_points_y[1], to_add,
                                       (len(y_train_)+2)*m,
                                       {'flag_lov': True})
        save_npy('am_half_size' + str(size) +'m' + str(m) + '.npy', half_ov)